# 🧭 **Introduction to Vector Search for Ramcharitmanas Book**

In this section, we will be using **LangChain** to implement vector search for our **Ramcharitmanas text**. The goal is to build a vector search system that can give us shlokas or padyas based on the semantic similarity of their meanings. To do this, we will utilize several powerful dependencies from the **LangChain ecosystem**.

### 🛠️ **Dependencies and Workflow**

We’re importing several key components that guide the process of building a vector database. Here’s a brief overview of how each dependency fits into our workflow:

1. **TextLoader (from `langchain_community.document_loaders`)**:
   - **Role**: This method helps load raw text into a format that can be easily processed by LangChain. For our padya finder, it will be used to load the meanings of padyas and convert them into a structured format for further processing.
   
2. **CharacterTextSplitter (from `langchain_text_splitters`)**:
   - **Role**: After loading the raw padya meanings, the `CharacterTextSplitter` method is used to split long documents into smaller, manageable chunks. In our case, each chunk will represent a single padya meaning, but in different contexts, this splitter can divide documents based on character count or other strategies. It’s a flexible tool to help organize larger text data.
   
3. **OpenAIEmbeddings (from `langchain_openai`)**:
   - **Role**: The `OpenAIEmbeddings` method will be used to convert each chunk of text (the padya meaning) into vector embeddings. These embeddings are numerical representations of the text, capturing the semantic meaning of each of the padya meanings. By leveraging **OpenAI's models**, we will ensure that our vectors are well-represented and can be easily compared to one another for similarity.
   
4. **Chroma (from `langchain_chroma`)**:
   - **Role**: Once we have the embeddings, we need to store them in a database for efficient retrieval. **Chroma** is a widely-used open-source vector database that will allow us to store and manage the embeddings. It supports efficient similarity searches, which is critical for building the recommendation system in our app. LangChain provides easy integration with Chroma and other vector databases, offering a flexible solution for vector storage and retrieval.

### 🔄 **Workflow Overview**
1. **Text Loading**: Load the raw meanings using the **TextLoader**.
2. **Text Splitting**: Break down the text into smaller chunks using the **CharacterTextSplitter**.
3. **Embedding Generation**: Convert each chunk into vector embeddings using **OpenAIEmbeddings**.
4. **Vector Storage**: Store the embeddings in a vector database (we will use **Chroma**) for fast similarity search.

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

When working with sensitive information like **API keys**, it's important to keep them secure and not hard-code them into your codebase. The best practice is to store such sensitive information as **environment variables**, which can be easily accessed when needed. One convenient way to manage these environment variables in Python is by using the `dotenv` library.

The `dotenv` library allows you to store environment variables in a `.env` file, and then load them into your Python environment. This ensures that sensitive information, like API keys, is kept out of the code and can be easily configured without exposing it directly.

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
import pandas as pd
from tqdm import tqdm  # For progress bar

---

## **Reading the clean ramcharitmanas dataset 📥**

In [4]:
padyas = pd.read_csv('datasets/ramcharitmanas_df.csv')
padyas

,Kand,Verse,Meaning,Verse Type,Verse Count,Page Number,id,tagged_meaning
0,बालकाण्ड,वर्णानामर्थसंघानां रसानां छन्दसामपि।\nमड़लानां...,"अक्षरों, अर्थसमूहों, रसों, छन्‍्दों और मंगलों ...",श्लोक,१,१७,0,"0 अक्षरों, अर्थसमूहों, रसों, छन्‍्दों और मंगलो..."
1,बालकाण्ड,भवानीशड्ूरोौ वन्दे श्रद्धाविश्वासरूपिणौ।\nया...,श्रद्धा और विश्वास के स्वरूप श्रीपार्वतीजी और ...,श्लोक,२,१७,1,1 श्रद्धा और विश्वासके स्वरूप श्रीपार्वतीजी और...
2,बालकाण्ड,वन्दे बोधमयं नित्यं गुरु शड्डूररूपिणम्‌।\nयमाथ...,"ज्ञानमय, नित्य, शड्डूररूपी गुरु की मैं वन्दना ...",श्लोक,३,१७,2,"2 ज्ञानमय, नित्य, शड्डूररूपी गुरुकी मैं वन्दना..."
3,बालकाण्ड,सीतारामगुणग्रामपुण्यारण्यविहारिणौ।\nवन्दे विशु...,श्रीसीतारामजी के गुणसमूहरूपी पवित्र वन में विह...,श्लोक,४,१७,3,3 श्रीसीतारामजीके गुणसमूहरूपी पवित्र वनमें विह...
4,बालकाण्ड,उद्धवस्थितिसंहारकारिणीं क्लेशहारिणीम्‌।\nसर्वश...,"उत्पत्ति, स्थिति (पालन) और संहार करने वाली, क्...",श्लोक,५,१८,4,"4 उत्पत्ति, स्थिति (पालन) और संहार करनेवाली, क..."
...,...,...,...,...,...,...,...,...
6157,उत्तरकाण्ड,मो सम दीन न दीन हित तुम्ह समान रघुबीर।\nअस बिच...,हे श्रीरघुवीर ! मेरे समान कोई दीन नहीं है और आ...,दो०,१३० (क),१०४६,6157,6157 हे श्रीरघुवीर ! मेरे समान कोई दीन नहीं है...
6158,उत्तरकाण्ड,कामिहि नारि पिआरि जिमि लोभिहि प्रिय जिमि दाम।\...,जै से कामी को स्त्री प्रिय लगती है और लोभी को ...,दो०,१३० (ख),१०४६,6158,6158 जैसे कामीको स्त्री प्रिय लगती है और लोभीक...
6159,उत्तरकाण्ड,य त्पूर्व प्रभुणा कृतं सुकविना श्रीशम्भुना दुर...,श्रेष्ठ कवि भगवान्‌ श्रीशंकरजीने पहले जिस दुर्...,श्लोक,१,१०४६,6159,6159 श्रेष्ठ कवि भगवान्‌ श्रीशंकरजीने पहले जिस...
6160,उत्तरकाण्ड,पुण्यं पापहरं सदा शिवकरं विज्ञानभक्तिप्रदं।\nम...,"यह श्रीरामचरितमानस पुण्यरूप, पापों का हरण करने...",श्लोक,२,१०४६,6160,"6160 यह श्रीरामचरितमानस पुण्यरूप, पापोंका हरण ..."


----

## **Cleaning the verse meanings**

In [5]:
# Hindi postpositions
postpositions = [
    'का', 'की', 'के', 'में', 'से', 'पर', 'को', 'तक', 'बिना', 'साथ', 'वाले', 'वाली', 'वाला'
]

for idx in range(len(padyas['Meaning'])):
    meaning = padyas.loc[idx, 'Meaning']
    tokens = [word for word in meaning.split(' ') if len(word) >= 3]
    for token in tokens:
        for post in postpositions:
            if token.endswith(post) and len(token) > len(post):
                root = token[:-len(post)]
                padyas.loc[idx, 'Meaning'] = padyas.loc[idx, 'Meaning'].replace(token, root + ' ' + post)

In [6]:
padyas.loc[0, 'Meaning']

'अक्षरों, अर्थसमूहों, रसों, छन्\u200d्दों और मंगलों की करने वाली सरस्वतीजी और गणेशजी की मैं वन्दना करता हूँ॥'

---

## **Tagging Descriptions with Id for Seamless Filtering 📚🔍**

Tagging padya with **ID** acts as a unique, reliable identifier for each padya, streamlining the process of filtering and searching. Rather than relying on costly and error-prone string matching, ID offer a straightforward way to pinpoint specific padyas. This boosts efficiency and ensures consistency in identifying and retrieving data.

By using ID tags, we can:
- **Eliminate the need for string matching**, improving search speed and accuracy.
- **Ensure precision** in retrieving exact matches, avoiding ambiguities in descriptions.
- **Integrate easily** with external data sources like Goodreads or Amazon, enriching the data pipeline.

Incorporating IDs into the system enhances both performance and scalability, ensuring that the padya finder operates seamlessly and intelligently.

In [7]:
padyas['id'] = list(range(0, len(padyas)))

In [8]:
padyas.to_csv('datasets/ramcharitmanas_df.csv', index=False)

In [9]:
padyas['tagged_meaning'] = padyas['id'].astype(str) + ' ' + padyas['Meaning']

In [10]:
padyas['tagged_meaning'].head()

0    0 अक्षरों, अर्थसमूहों, रसों, छन्‍्दों और मंगलो...
1    1 श्रद्धा और विश्वास के स्वरूप श्रीपार्वतीजी औ...
2    2 ज्ञानमय, नित्य, शड्डूररूपी गुरु की मैं वन्दन...
3    3 श्रीसीतारामजी के गुणसमूहरूपी पवित्र वन में व...
4    4 उत्पत्ति, स्थिति (पालन) और संहार करने वाली, ...
Name: tagged_meaning, dtype: object

In [11]:
for i, row in padyas.iterrows():
    if '\n' in row['tagged_meaning']:
        padyas.loc[i, 'Meaning'] = padyas.loc[i, 'Meaning'].replace('\n', ' ')
        padyas.loc[i, 'tagged_meaning'] = padyas.loc[i, 'tagged_meaning'].replace('\n', ' ')
        
# print(sorted(list(set(all_chs))))

In [12]:
all_chs = []
for _, row in padyas.iterrows():
    for ch in row['tagged_meaning']:
        all_chs.append(ch)
print(sorted(list(set(all_chs))))

[' ', '!', '"', "'", '(', ')', '*', ',', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', '?', '[', ']', 'ँ', 'ं', 'ः', 'अ', 'आ', 'इ', 'ई', 'उ', 'ऊ', 'ऋ', 'ए', 'ऐ', 'ओ', 'औ', 'क', 'ख', 'ग', 'घ', 'च', 'छ', 'ज', 'झ', 'ञ', 'ट', 'ठ', 'ड', 'ढ', 'ण', 'त', 'थ', 'द', 'ध', 'न', 'प', 'फ', 'ब', 'भ', 'म', 'य', 'र', 'ल', 'व', 'श', 'ष', 'स', 'ह', '़', 'ऽ', 'ा', 'ि', 'ी', 'ु', 'ू', 'ृ', 'े', 'ै', 'ॉ', 'ो', 'ौ', '्', 'ॐ', '॑', '॒', '।', '॥', '०', '१', '२', '३', '४', '५', '६', '\u200c', '\u200d', '“', '”']


---

## **Saving Tagged Descriptions to a Text File for Use with TextLoader 📝**

The **TextLoader** method from LangChain is designed to work with raw text files, not directly with a **Pandas DataFrame**. Since the book descriptions are stored in the DataFrame, we need to extract and save them in a compatible format.

To bridge this gap, we need to:
1. **Extract tagged meanings** (with IDs) from the DataFrame.
2. **Save them to a text file**, where each description is stored in a separate line.

This process ensures that the **TextLoader** can properly read the meanings and work with the LangChain framework. Let's save our tagged descriptions and get them ready for the next steps in building the vector database.

In [13]:
tagged_meanings = list(padyas['tagged_meaning'])
tagged_meanings = [(meaning + '\n') for meaning in tagged_meanings]
tagged_meanings[:5]

['0 अक्षरों, अर्थसमूहों, रसों, छन्\u200d्दों और मंगलों की करने वाली सरस्वतीजी और गणेशजी की मैं वन्दना करता हूँ॥\n',
 '1 श्रद्धा और विश्वास के स्वरूप श्रीपार्वतीजी और श्रीशड्डरजी की मैं वन्दना करता हूँ, जिन के बिना सिद्धजन अपने अन्तःकरण में स्थित ईश्वर को नहीं देख सकते॥\n',
 '2 ज्ञानमय, नित्य, शड्डूररूपी गुरु की मैं वन्दना करता हूँ, जिन के आश्रित होने से ही टेढ़ा चन्द्रमा भी सर्वत्र वन्दित होता है॥\n',
 '3 श्रीसीतारामजी के गुणसमूहरूपी पवित्र वन में विहार करनेवाले, विशुद्ध विज्ञानसम्पन्न कवीश्वर श्रीवाल्मीकिजी और कपीश्वर श्रीहनुमानूजी की मैं वन्दना करता हूँ॥\n',
 '4 उत्पत्ति, स्थिति (पालन) और संहार करने वाली, क्लेशों की हरने वाली तथा सम्पूर्ण कल्याणों की करने वाली श्रीरामचन्द्रजी की प्रियतमा श्रीसीताजी को मैं नमस्कार करता हूँ॥\n']

In [14]:
with open('datasets/tagged_meanings.txt', 'w') as file:
    file.writelines(tagged_meanings)

In [15]:
raw_documents = TextLoader('datasets/tagged_meanings.txt').load()

## **Splitting the Text into Chunks ✂️**

Now that we have our tagged meanings saved into a text file, the next step is **splitting** the text into manageable pieces using LangChain’s `CharacterTextSplitter`.

In [16]:
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator='\n')
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 104, which is longer than the specified 0
Created a chunk of size 151, which is longer than the specified 0
Created a chunk of size 124, which is longer than the specified 0
Created a chunk of size 155, which is longer than the specified 0
Created a chunk of size 165, which is longer than the specified 0
Created a chunk of size 348, which is longer than the specified 0
Created a chunk of size 210, which is longer than the specified 0
Created a chunk of size 170, which is longer than the specified 0
Created a chunk of size 188, which is longer than the specified 0
Created a chunk of size 167, which is longer than the specified 0
Created a chunk of size 188, which is longer than the specified 0
Created a chunk of size 180, which is longer than the specified 0
Created a chunk of size 216, which is longer than the specified 0
Created a chunk of size 225, which is longer than the specified 0
Created a chunk of size 234, which is longer than the specified 0
Created a 

In [17]:
len(documents)

6162

- **`separator='\n'`** tells the splitter to break the text **at each newline** — perfect, because we saved each padya meaning on a new line.
- **`chunk_size=0`** means **do not group multiple meanings together**; keep each meaning as its own chunk (prioritize splitting over separator).
- **`chunk_overlap=0`** means **no overlap** between chunks; every meaning stays cleanly separated without repeating parts.

This setup keeps each meaning neatly isolated and ready for embedding. 🚀

In [18]:
documents[0]

Document(metadata={'source': 'datasets/tagged_meanings.txt'}, page_content='0 अक्षरों, अर्थसमूहों, रसों, छन्\u200d्दों और मंगलों की करने वाली सरस्वतीजी और गणेशजी की मैं वन्दना करता हूँ॥')

---

## 🧠 Building Our Vector Database with Chroma

Now that we have our padya meanings split and ready, the next step is to **embed** them and **store** them in a **vector database**.  
For this, we're using **Chroma**, a lightweight, open-source vector database that's very popular for working with document embeddings in LangChain.

Chroma allows us to efficiently store and search embeddings locally — no need for external services — making it perfect for prototyping apps like our padya finder.

### 🔐 API Key Setup: Don't Forget the `.env` File

Since we're making API calls to OpenAI, we need to **authenticate** using our API key.

Create a `.env` file in your project root with the following content:

```
OPENAI_API_KEY=your_openai_api_key_here
```

And don't forget to **load the environment variables** before using `OpenAIEmbeddings`

> 📦 `.env` keeps sensitive information secure and out of your main codebase.





In [19]:
if not os.path.exists('datasets/chroma_db'):
    os.mkdir('datasets/chroma_db')

In [20]:
embedding_model = OpenAIEmbeddings(model='text-embedding-3-large')

db_padyas = Chroma(
    collection_name='padya_meanings',
    embedding_function=embedding_model,
    persist_directory='datasets/chroma_db',
)

batch_size = 50

for i in tqdm(range(0, len(documents), batch_size)):
    # print(i)
    batch = documents[i: i+batch_size]
    db_padyas.add_documents(batch)

100%|█████████████████████████████████████████| 124/124 [04:50<00:00,  2.34s/it]


In [21]:
db_padyas._collection.count() 

6162

### 🚨 Why Do We Need Batching?

When using models like OpenAI's `text-embedding-ada-002`, **there are strict rate limits** on how many tokens you can send per minute.  
If we try to embed a large number of documents at once, we'll hit a **RateLimitError**.

> ✅ **Batching** helps us process smaller, manageable chunks of data without exceeding API rate limits — making the upload safe, stable, and reliable.



### 💾 How Chroma Persists Data Locally

When you use Chroma with a `persist_directory` (like `datasets/chroma_db`), it saves your vector database **on disk** automatically.



In [22]:
root_dir = 'datasets/chroma_db/'

# Create a list to store file info
file_data = []

# Walk through the directory
for foldername, subfolders, filenames in os.walk(root_dir):
    for filename in filenames:
        file_path = os.path.join(foldername, filename)
        size_bytes = os.path.getsize(file_path)
        file_data.append({
            'File Name': filename,
            'File Path': file_path,
            'Size (KB)': round(size_bytes / 1024, 2),
        })

# Convert to a pandas DataFrame for a clean table
df_files = pd.DataFrame(file_data)

df_files

,File Name,File Path,Size (KB)
0,chroma.sqlite3,datasets/chroma_db/chroma.sqlite3,28972.00
1,data_level0.bin,datasets/chroma_db/3ee9331c-32d8-4850-8fb8-9b6...,72820.31
2,length.bin,datasets/chroma_db/3ee9331c-32d8-4850-8fb8-9b6...,23.44
3,link_lists.bin,datasets/chroma_db/3ee9331c-32d8-4850-8fb8-9b6...,50.93
4,header.bin,datasets/chroma_db/3ee9331c-32d8-4850-8fb8-9b6...,0.10
5,index_metadata.pickle,datasets/chroma_db/3ee9331c-32d8-4850-8fb8-9b6...,337.92


#### 📦 Chroma DB Folder Contents

| File Name | Description |
|:---|:---|
| `chroma.sqlite3` | The main SQLite database that stores metadata about documents and embeddings. | 
| `data_level0.bin` | Binary file containing the raw vector embeddings stored efficiently for fast search. | 
| `length.bin` | Stores the lengths of each vector entry, used to efficiently index into `data_level0.bin`. | 
| `link_lists.bin` | Contains graph link information for ANN (Approximate Nearest Neighbors) search structure. | 
| `header.bin` | Small file that stores metadata about how the binary data is organized (dimensions, format). | 
| `index_metadata.pickle` | Pickle file that contains additional metadata about the vector index for Chroma's internal use. | 


---

## 📝 Reading the saved embeddings directly

Load the Chroma database from the specified `persist_directory` where it was saved. Also, initialize the embedding model you used to save the embeddings, such as `OpenAIEmbeddings`.

> **No Need to run the previous section to create embeddings in that case

In [23]:
embedding_model = OpenAIEmbeddings(model='text-embedding-3-large')

db_padyas = Chroma(
    collection_name='padya_meanings',
    embedding_function=embedding_model,
    persist_directory='datasets/chroma_db',
)

In [24]:
db_padyas._collection.count() 

6162

## 🔍 Querying Saved Embeddings

After saving embeddings into the Chroma database, you can query the stored data to find similar padyas, or search for content related to a specific topic. Below are the steps to effectively query the Chroma database.

### ~🔍 **Perform a Similarity Search**

Once the database is loaded, you can query it by providing a textual query. The database will return the most similar padyas based on the stored embeddings. Here's how you can do a similarity search:

In [25]:
query = "Hanuman Ji meets Mata Sita"
docs = db_padyas.similarity_search(query=query, k = 10)
docs

[Document(id='e3856e3b-fec9-4456-84bd-3bfcdc7e93c2', metadata={'source': 'datasets/tagged_meanings.txt'}, page_content='2482 उन्होंने अपने श्रीमुख से सीताजी, लक्ष्मणजी और सखा गुह को तीर्थराज की महिमा कहकर सुनायी। तदनन्तर प्रणाम करके, वन और बगीचों को देखते हुए और बड़े प्रेम से माहात्म्य कहते हुए--॥'),
 Document(id='3e825e68-2d94-41d0-a4e0-3e6a7c3334b0', metadata={'source': 'datasets/tagged_meanings.txt'}, page_content='4225 हनुमानजी के प्रेमयुक्त वचन सुनकर सीताजी के मन में विश्वास उत्पन्न हो गया। उन्होंने जान लिया  कि यह मन, वचन और कर्म से कृपासागर श्रीरघुनाथजी का दास है॥'),
 Document(id='64ac5ab7-ca45-41bf-90fe-887e5635e892', metadata={'source': 'datasets/tagged_meanings.txt'}, page_content='2315 वे हर्षित हृदय से माता सुमित्राजी के पास आये, मानो अंधा फिर से नेत्र पा गया हो। उन्होंने जाकर माता के चरणों में मस् तक नवाया। किन्तु उन का मन रघुकुल को आनन्द देने वाले श्रीरामजी और जानकीजी के साथ था॥'),
 Document(id='b8b0c796-e752-45bb-86e6-1a2696e6dd00', metadata={'source': 'datasets/tagged_m

### 📚 **Fetching IDs of Similar Meanings**

After retrieving similar meanings from Chroma, each meaning's content begins with the corresponding ID (since we tagged meanings with IDs earlier). To fetch full padya details like Verse, Kand, and Verse Type, we can extract the ID from the retrieved document and query our original `padyas` DataFrame.

In [26]:
padyas[padyas['id'] == int(docs[0].page_content.split()[0].strip())]

,Kand,Verse,Meaning,Verse Type,Verse Count,Page Number,id,tagged_meaning
2482,अयोध्याकाण्ड,कहि सिय लखनहि सखहि सुनाई। श्रीमुख तीरथराज बड़ा...,"उन्होंने अपने श्रीमुख से सीताजी, लक्ष्मणजी और ...",चौपाई,२,४३१,2482,"2482 उन्होंने अपने श्रीमुख से सीताजी, लक्ष्मणज..."


Here’s what happens:
- `docs[0].page_content.split()[0].strip()` extracts the first word (the ISBN) from the document content.
- We convert it to an integer to match the `isbn13` column in the `books` DataFrame.
- Finally, we filter the DataFrame to get full book details for the matching ISBN.

### 🔍 **Combining Querying Functionality into a Function**

To make our semantic search workflow cleaner and reusable, we wrap the querying and retrieval logic into a single function: `retrieve_semantic_recommendations`.

Here’s what this function does:
- Takes a **query** string (what the user is searching for) and a **top_k** (how many results to return).
- Uses the Chroma database to perform a **similarity search** based on the query.
- Extracts the **ISBNs** from the matched documents.
- Looks up the corresponding book details from the `books` DataFrame.
- Returns a DataFrame of recommended books!

Now, with just one line, you can generate smart, semantic book recommendations based on any user query! 📚✨

In [27]:
def retrieve_semantic_recommendations(
    query: str,
    top_k: int = 10
) -> pd.DataFrame:
    recommendations = db_padyas.similarity_search(query=query, k = top_k)
    
    padyas_list = []
    for i in range(0, len(recommendations)):
        padyas_list += [int(recommendations[i].page_content.strip('"').split()[0].strip())]
        
    return padyas[padyas['id'].isin(padyas_list)]

In [28]:
retrieve_semantic_recommendations(
    'हनुमान जी की सीता सीताजी से भेन्ट',
    10
)

,Kand,Verse,Meaning,Verse Type,Verse Count,Page Number,id,tagged_meaning
1432,बालकाण्ड,सोहति सीय राम कै जोरी। छबि सिंगारु मनहुँ एक ठो...,श्रीसीता-रामजी की जोड़ी ऐसी सुशोभित हो रही है ...,चौपाई,४,२५६,1432,1432 श्रीसीता-रामजी की जोड़ी ऐसी सुशोभित हो रह...
2566,अयोध्याकाण्ड,आगें रामु लखनु बने पाछें। तापस बेष बिराजत काछे...,"आगे श्रीरामजी हैं, पीछे लक्ष्मणजी सुशोभित हैं।...",चौपाई,१,४४५,2566,"2566 आगे श्रीरामजी हैं, पीछे लक्ष्मणजी सुशोभित..."
3395,अयोध्याकाण्ड,कहति न सीय सकुचि मन माहीं। इहाँ बसब रजनीं भल न...,"सीताजी कुछ कहती नहीं हैं, परन्तु मन में सकुचा ...",चौपाई,४,५८४,3395,"3395 सीताजी कुछ कहती नहीं हैं, परन्तु मन में स..."
3551,अयोध्याकाण्ड,लखनहि भेंटि प्रनामु करि सिर धरि सिय पद धूरि।\n...,फिर लक्ष्मणजी को क्रमशः भेंटकर तथा प्रणाम कर क...,दो०,३१८,६१०,3551,3551 फिर लक्ष्मणजी को क्रमशः भेंटकर तथा प्रणाम...
4193,सुन्दरकाण्ड,देखि मनहि महुँ कीन्ह प्रनामा। बैठेहिं बीति जात...,सीताजी को देखकर हनुमानूजीने उन्हें मनही में प्...,चौपाई,४,७२२,4193,4193 सीताजी को देखकर हनुमानूजीने उन्हें मनही म...
4224,सुन्दरकाण्ड,नर बानरहि संग कहु कैसें। कही कथा भइ संगति जैसें॥,[सीताजीने पूछा--] नर और वानर का सज्ज कहो कै से...,चौपाई,६,७२६,4224,4224 [सीताजीने पूछा--] नर और वानर का सज्ज कहो ...
4225,सुन्दरकाण्ड,कपि के बचन सप्रेम सुनि उपजा मन बिस्वास।\nजाना ...,हनुमानजी के प्रेमयुक्त वचन सुनकर सीताजी के मन ...,दो०,१३,७२७,4225,4225 हनुमानजी के प्रेमयुक्त वचन सुनकर सीताजी क...
4242,सुन्दरकाण्ड,सीता मन भरोस तब भयऊ। पुनि लघु रूप पवनसुत लयऊ॥,तब (उ से देखकर) सीताजी के मन में विश्वास हुआ। ...,चौपाई,५,७२९,4242,4242 तब (उ से देखकर) सीताजी के मन में विश्वास ...
4244,सुन्दरकाण्ड,मन संतोष सुनत कपि बानी। भगति प्रताप तेज बल सान...,"भक्ति, प्रताप, तेज और बल से सनी हुई हनुमानूजी ...",चौपाई,१,७३०,4244,"4244 भक्ति, प्रताप, तेज और बल से सनी हुई हनुमा..."
5176,लंकाकाण्ड,दूरिहि ते प्रनाम कपि कीन्हा। रघुपति दूत जानकी ...,हनुमानूजीने [सीताजीको] दूर से ही प्रणाम किया। ...,चौपाई,३,८८५,5176,5176 हनुमानूजीने [सीताजीको] दूर से ही प्रणाम क...


In [29]:
import re
verse = padyas.loc[506, 'Verse']
verse_parts = re.split(r'(।|॥)', verse)
verse_str = '\n'.join([verse_parts[i] + verse_parts[i + 1] for i in range(0, len(verse_parts) - 1, 2)])
verse_str

'जोगी अकंटक भए पति गति सुनत रति मुरुछित भई।\n\nरोदति बदति बहु भाँति करुना करति संकर पहिं गई॥\n\nअति प्रेम करि बिनती बिबिध बिधि जोरि कर सन्मुख रही।\n\nप्रभु आसुतोष कृपाल सिव अबला निरखि बोले सही॥'